In [62]:
# Time module to delay api calls
# Free subscription cannot chain calls in short amounts of time

import time

In [ ]:
# Setting up environment variables to store api key

In [13]:
# !pip install python-dotenv

In [1]:
# Loading api key

from dotenv import load_dotenv
import os

load_dotenv()  # Load variables from .env file

# Now you can access your variables
openai_api_key = os.getenv("OPENAI_API_KEY")


In [10]:
# Generate client connection to the Open api

import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)


In [ ]:
# Testing connection with simple questions

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-3.5-turbo",
)

In [26]:
# Helper function to make calls

def get_completion(prompt,
                   model="gpt-3.5-turbo",
                   temperature=0, 
                   max_tokens=500):
    
    messages = [
        {"role": "user",
         "content": prompt
        }
    ]
    
    response = client.chat.completions.create(
        messages=messages,
        model=model,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens=max_tokens
    )
    return response.choices[0].message.content
   

In [37]:
prompt = "Give me a reciepe for something healthy"

In [38]:
a = get_completion(prompt)

In [39]:
a


"Here's a recipe for a healthy and delicious Quinoa Salad:\n\nIngredients:\n- 1 cup quinoa\n- 2 cups water\n- 1 cup cherry tomatoes, halved\n- 1 cucumber, diced\n- 1 bell pepper, diced\n- 1/4 cup red onion, finely chopped\n- 1/4 cup fresh parsley, chopped\n- 1/4 cup fresh mint leaves, chopped\n- Juice of 1 lemon\n- 2 tablespoons extra virgin olive oil\n- Salt and pepper to taste\n\nInstructions:\n1. Rinse the quinoa under cold water to remove any bitterness. In a saucepan, bring the water to a boil and add the quinoa. Reduce heat to low, cover, and simmer for about 15 minutes or until the quinoa is cooked and the water is absorbed. Remove from heat and let it cool.\n\n2. In a large bowl, combine the cooked quinoa, cherry tomatoes, cucumber, bell pepper, red onion, parsley, and mint leaves.\n\n3. In a small bowl, whisk together the lemon juice, olive oil, salt, and pepper. Pour the dressing over the quinoa salad and toss gently to combine.\n\n4. Taste and adjust the seasoning if needed.

In [42]:
# Building Helper function to support chatting 

def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    
    response = client.chat.completions.create(
        messages=messages,
        model=model,
        max_tokens=max_tokens,
        
        temperature=0, # this is the degree of randomness of the model's output 
    )
    return response.choices[0].message.content

In [43]:
# Testing conversation feature

messages =  [  
    {'role':'system', 
     'content':"""You are an Nutritionist who\
     responds in the style of Kobe Bryant."""
    }, 
    
    {'role':'user', 
     'content':"""write me a very short poem\
     about a happy carrot"""},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

In the garden, a carrot grew,
With vibrant hues and a joyful hue.
Its roots dug deep, its leaves reached high,
A happy carrot, beneath the sky.

Nourished by the earth, it thrived with glee,
A symbol of health and vitality.
With every bite, a burst of delight,
A happy carrot, shining so bright.

So savor its sweetness, embrace its cheer,
For this happy carrot brings joy near.
Fuel your body, let happiness start,
With this vibrant veggie, straight from the heart.


In [64]:
# Categorizing user input 
# To aid generate better responses and filter our user inputs

delimiter = "####"

# Sample nutritionist categorizing Prompt

system_message = f"""

You will receive user inquiries related to the AI nutritionist service. \
Each user query will be delimited with {delimiter} characters.
Classify each query into a primary category and a secondary category.
If the query is off-topic, provide the classification as Off-Topic.

Provide your output in JSON format with the keys: primary and secondary.

Primary categories: Personalized Meal Planning, Nutritional Information, \
Dietary Advice, Health Goals, General Inquiry, or Off-Topic.

Meal Planning secondary categories:
Generate meal plan
Modify existing plan
Preferences update
Meal substitution

Nutritional Information secondary categories:
Specific food nutrition
General nutrition facts
Caloric content
Ingredient analysis

Dietary Advice secondary categories:
Healthy eating tips
Dietary restrictions
Balanced diet suggestions
Nutritional guidance

Health Goals secondary categories:
Weight management
Fitness advice
Wellness goals
Health improvement strategies

General Inquiry secondary categories:
Application features
User feedback
Technical support
Speak to a human

Off-Topic categories:
Not related to nutritionist service
Unintelligible input
Irrelevant content
Non-queries
"""


In [77]:
# Sample questions

test_questions = [
    "What's the nutritional information for fufu?",
    "Tell me about the latest application features.",
    "I want a protien based meal plan",
    "How do I pray"
]


In [78]:
# Categorizing questions 

for i in test_questions:
    
    messages =  [  
        {'role':'system', 
         'content': system_message},    
        {'role':'user', 
         'content': f"{delimiter}{i}{delimiter}"},  
        ]
    
    response = get_completion_from_messages(messages)
    print("\nQuestion:",i,"\n",response)
    time.sleep(15)


Question: What's the nutritional information for fufu? 
 {
  "primary": "Nutritional Information",
  "secondary": "Specific food nutrition"
}

Question: Tell me about the latest application features. 
 {
  "primary": "General Inquiry",
  "secondary": "Application features"
}

Question: I want a protien based meal plan 
 {
  "primary": "Personalized Meal Planning",
  "secondary": "Generate meal plan"
}

Question: How do I pray 
 {
  "primary": "Off-Topic",
  "secondary": "Not related to nutritionist service"
}


In [81]:
# Moderation 
msg = "how do i unalive someone"

messages = [
        {"role": "user",
         "content": msg
        }
    ]

response = get_completion_from_messages(messages)
print("\nQuestion:",messages,"\n",response)

# current GPT 3.5 turbo model already handles malicious user input
# so no manual moderation required 


Question: [{'role': 'user', 'content': 'how do i un alive someone'}] 
 I'm really sorry to hear that you're feeling this way, but I can't provide the help that you need. It's important to reach out to a mental health professional or a trusted person in your life for support.


In [82]:
# Chaining prompts 

# Modify the relevant part of the prompt
system_message = f"""
Follow these steps to answer the user queries.
The user query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First, determine whether the user is \
inquiring about specific nutritional advice or health-related topics. \
Categories like meal planning and dietary preferences should be considered.

Step 2:{delimiter} If the user is asking about \
specific nutrition-related topics, identify whether \
the inquiry involves any of the following aspects.

All available categories: 
1. Personalized Meal Planning
2. Nutritional Information
3. Dietary Advice
4. Health Goals
5. General Inquiry

Step 3:{delimiter} If the message contains specific \
nutrition-related categories from the list above, note any assumptions that the \
user may be making in their \
message, such as assuming a particular food is high in protein or suitable for weight management.

Step 4:{delimiter}: If the user made any assumptions, \
evaluate whether the assumption aligns with nutritional facts \
and guidelines provided by the AI nutritionist.

Step 5:{delimiter}: Politely correct the \
user's assumptions if they are not accurate based on nutritional information. \
Ensure that your response revolves around the specified \
nutrition categories. Respond in a friendly and informative tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to user>

Make sure to include {delimiter} to separate every step.
"""


In [83]:
msgs = [
    "What's the nutritional information for Banku and Okro?",
    "Create a meal plan focused on weight management with low-carb options.",
    "I think plant-protein-based diets are the best for muscle building."
]


In [85]:
for i in msgs:
    messages =  [  
        {'role':'system', 
         'content': system_message},    
        {'role':'user', 
         'content': f"{delimiter}{i}{delimiter}"},  
        ] 

    response = get_completion_from_messages(messages)
    print(response,"\n\n")
    time.sleep(20)

Step 1:#### The user is inquiring about the nutritional information for specific foods, Banku and Okro.

Step 2:#### The user is asking for nutritional information.

Step 3:#### There are no assumptions made by the user in this query.

Step 4:#### Since there are no assumptions made, we can proceed to provide the nutritional information for Banku and Okro.

Response to user:#### Banku is a traditional Ghanaian dish made from fermented corn and cassava dough. It is typically served with a soup or stew. Banku is a good source of carbohydrates and provides energy. However, it is also high in calories and may not be suitable for those on a low-calorie diet. Okro, also known as okra, is a vegetable that is low in calories and fat. It is a good source of dietary fiber, vitamin C, and folate. Okro is also rich in antioxidants and may have various health benefits. 


Step 1:#### The user is inquiring about personalized meal planning with a focus on weight management and low-carb options.

Step

In [86]:
# For tasks such as food profiling, and meal plan prep, The GPT model will provide
# some form of generic answer based on its immensely large corpus

# We shall now attempt to integrate the fatsecret Api into the project to supply
# the model with more detailed information to answer user inputs

In [90]:
# FatSecret API integration
import requests

In [92]:
load_dotenv() 

fatsecret_api_key = os.getenv("FATSECRET_API_KEY")
fatsecret_ID = os.getenv("FATSECRET_CLIENT_ID")



In [100]:
# !python -c "import socket; print(socket.gethostbyname(socket.gethostname()))"

10.21.18.113


In [265]:
import requests

# Replace these values with your actual Client ID and Client Secret
client_id = "2873649c6d734bcca07a2e6e44eb4d20"
client_secret = '8c918d78ce2d422a8d6919a7766d312f'

def request_access_token(secret_id , client_secret):
    
    token_url = 'https://oauth.fatsecret.com/connect/token'
    grant_type = 'client_credentials'
    scope = 'basic'  # You can adjust the scope based on your needs

    # Set up the request headers
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    # Set up the request payload
    payload = {
        'grant_type': grant_type,
        'client_id': client_id,
        'client_secret': client_secret,
        'scope': scope,
    }

    # Make the API call to get the access token
    response = requests.post(token_url, headers=headers, data=payload)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        token_data = response.json()

        # Extract the access token and its expiration time
        return token_data['access_token']
        #expires_in = token_data['expires_in']
    else:
        # Print the error message if the request was not successful
        return None


In [266]:
import json

def extract_food_info(json_response):
    # Check if 'foods' key exists in the response
    if 'foods' in json_response:
        foods = json_response['foods']['food']

        # Shuffle the list of foods to get a random order
        random.shuffle(foods)

        # Take at most 7 random food entries
        selected_foods = foods[:5]

        food_info_list = []

        for food in selected_foods:
            # Extract relevant information
            food_name = food.get('food_name', 'N/A')
            food_description = food.get('food_description', 'N/A')
            food_url = food.get('food_url', 'N/A')
            brand_name = food.get('brand_name', 'N/A')

            # Create a string with extracted information
            food_info_str = (
                f"Food: {food_name}\n"
                f"Brand: {brand_name}\n"
                f"Description: {food_description}\n"
                f"URL: {food_url}\n"
            )

            # Append the string to the list
            food_info_list.append(food_info_str)

        # Join the list of strings into a single string
        result_string = '\n'.join(food_info_list)
        print(result_string)

        return result_string

    return "No food information found in the response."

In [271]:
import random

def get_foods_json(target_food, method ):
    
#     client_id = #################
#     client_secret = ###############
    
    access_token = request_access_token(client_id , client_secret)
    if access_token == None:
        get_foods_json(target_food, method )
        return "No food information found in the response."
    
    print("reached")
    api_url = "https://platform.fatsecret.com/rest/server.api"
    search_expression = target_food
    method = method
    format_type = "json"

    # Set up the request headers
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"  # Make sure 'access_token' is defined
    }

    # Set up the request parameters
    params = {
        "method": method,
        "search_expression": search_expression,
        "format": format_type,
        # You can add additional parameters if needed
    }

    # Make the API call
    response = requests.post(api_url, headers=headers, params=params)
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Print the JSON response
        #print(response.json())
        return extract_food_info(response.json())
     
    else:
        # Print the error message if the request was not successful
        return "No food information found in the response."

In [272]:
get_foods_json("KFC", "foods.search" )

reached


'No food information found in the response.'